# Create Reference Filemaker

In [1]:
from fsspec_reference_maker.hdf import SingleHdf5ToZarr 
from fsspec_reference_maker.combine import MultiZarrToZarr

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import zipfile
import logging
import fsspec
import ujson
from tqdm import tqdm
from glob import glob
import intake
import cftime
import os

In [3]:
from ncar_jobqueue import NCARCluster
from dask.distributed import Client
import dask

cluster = NCARCluster(memory='10 GB')
cluster.scale(30)

client = Client(cluster)
client

/glade/u/home/mgrover/.local/lib/python3.9/site-packages/dask_jobqueue/core.py:19: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/glade/u/home/mgrover/.local/lib/python3.9/site-packages/dask_jobqueue/core.py:19: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/glade/u/home/mgrover/.local/lib/python3.9/site-packages/dask_jobqueue/htcondor.py:6: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import parse_bytes


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.36:37114,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
catalog = intake.open_esm_datastore('/glade/work/mgrover/intake-esm-catalogs/silver-linings-test.json')

In [15]:
files = sorted(catalog.search(frequency='month_1', variable='T').df.path)

In [16]:
for file in files:
    print(file)

/glade/campaign/cesm/development/wawg/WACCM6-TSMLT-GEO/SAI1/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.001/atm/proc/tseries/month_1/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.001.cam.h0.T.203501-206912.nc
/glade/campaign/cesm/development/wawg/WACCM6-TSMLT-GEO/SAI1/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.002/atm/proc/tseries/month_1/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.002.cam.h0.T.203501-206912.nc
/glade/campaign/cesm/development/wawg/WACCM6-TSMLT-GEO/SAI1/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.003/atm/proc/tseries/month_1/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.003.cam.h0.T.203501-206912.nc
/glade/campaign/cesm/development/wawg/WACCM6-TSMLT-GEO/SAI1/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.004/atm/proc/tseries/month_1/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.004.cam.h0.T.203501-206912.nc
/glade/campaign/cesm/development/wawg/WACCM6-TSMLT-GEO/SAI1/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.005/atm/proc/tseries/month_1/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEF

In [7]:
def gen_json(u):
    so = dict(
        mode="rb", anon=True, default_fill_cache=False, default_cache_type="none"
    )
    with fsspec.open(u, **so) as inf:
        h5chunks = SingleHdf5ToZarr(inf, u, inline_threshold=300)
        with open(f"jsons/{u.split('/')[-1]}.json", 'wb') as outf:
            outf.write(ujson.dumps(h5chunks.translate()).encode())

In [8]:
# Create json/ folder if it doesn't already exist
import pathlib
pathlib.Path('./jsons/').mkdir(exist_ok=True)

In [9]:
dask.compute(*[dask.delayed(gen_json)(file) for file in files]);

In [6]:
json_list = sorted(glob("./jsons/*.json"))[:4]

In [25]:
m_list = []
for j in tqdm(json_list):
    with open(j) as f:
        m_list.append(fsspec.get_mapper("reference://", 
                        fo=ujson.load(f),
                        remote_protocol='s3',
                        remote_options={'anon':True}))

100%|██████████| 4/4 [00:00<00:00,  5.72it/s]


In [26]:
m_list

In [36]:
%%time
ds1 = xr.open_mfdataset(m_list, engine='zarr', combine='nested', concat_dim='time', 
                        coords='minimal', data_vars='minimal', compat='override', decode_times=False,
                        parallel=True)
ds1

CPU times: user 1.34 s, sys: 159 ms, total: 1.5 s
Wall time: 2.51 s


<xarray.Dataset>
Dimensions:       (time: 12775, lev: 70, lat: 192, lon: 288, ilev: 71, nbnd: 2)
Coordinates:
  * ilev          (ilev) float64 nan nan nan nan nan nan ... nan nan nan nan nan
  * lat           (lat) float64 nan nan nan nan nan nan ... nan nan nan nan nan
  * lev           (lev) float64 nan nan nan nan nan nan ... nan nan nan nan nan
  * lon           (lon) float64 nan nan nan nan nan nan ... nan nan nan nan nan
  * time          (time) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... nan nan nan nan nan
Dimensions without coordinates: nbnd
Data variables: (12/26)
    P0            float64 ...
    T             (time, lev, lat, lon) float32 dask.array<chunksize=(1, 35, 96, 144), meta=np.ndarray>
    ch4vmr        (time) float64 dask.array<chunksize=(512,), meta=np.ndarray>
    co2vmr        (time) float64 dask.array<chunksize=(512,), meta=np.ndarray>
    date          (time) float64 dask.array<chunksize=(1024,), meta=np.ndarray>
    date_written  (time) object dask.array<chunksize=(1,), meta=np.ndarray>
    ...            ...
    nsbase        float64 ...
    nscur         (time) float64 dask.array<chunksize=(1024,), meta=np.ndarray>
    nsteph        (time) float64 dask.array<chunksize=(1024,), meta=np.ndarray>
    sol_tsi       (time) float64 dask.array<chunksize=(512,), meta=np.ndarray>
    time_bnds     (time, nbnd) float64 dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_written  (time) object dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    _NCProperties:     version=2,netcdf=4.6.3,hdf5=1.10.5
    _nc3_strict:       1
    case:              b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.001
    host:              cheyenne5
    initial_file:      b.e21.BWSSP245cmip6.f09_g17.CMIP6-SSP2-4.5-WACCM.001.c...
    logname:           geostrat
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    source:            CAM
    time_period_freq:  day_1
    topography_file:   /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/fv_0.9x1...

In [39]:
!rm -r test.zarr

In [40]:
ds1[['T']].to_zarr('test.zarr')

KeyError: "('open_dataset-concatenate-concatenate-dcb5aba3fd64eea23ddb6d5dc78cba3b', 12028, 0, 0, 0)"

In [ ]:
ds1 = xr.open_zarr('test.zarr)

In [28]:
%%time
ds1.mean(dim='time').compute()

CPU times: user 25.4 s, sys: 1.73 s, total: 27.2 s
Wall time: 40.3 s


<xarray.Dataset>
Dimensions:    (lev: 70, lat: 192, lon: 288, ilev: 71, nbnd: 2)
Coordinates:
  * ilev       (ilev) float64 nan nan nan nan nan nan ... nan nan nan nan nan
  * lat        (lat) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan
  * lev        (lev) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan
  * lon        (lon) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan
Dimensions without coordinates: nbnd
Data variables: (12/24)
    P0         float64 1e+05
    T          (lev, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan
    ch4vmr     float64 nan
    co2vmr     float64 nan
    date       float64 nan
    datesec    float64 0.0
    ...         ...
    ndcur      float64 nan
    nsbase     float64 0.0
    nscur      float64 0.0
    nsteph     float64 3.365e+05
    sol_tsi    float64 -1.0
    time_bnds  (nbnd) float64 6.386e+03 6.387e+03

In [29]:
%%time
ds2 = xr.open_mfdataset(files, engine='netcdf4', combine='nested', concat_dim='time', 
                        coords='minimal', data_vars='minimal', compat='override',
                        parallel=True, chunks={'time':24})
ds2

CPU times: user 124 ms, sys: 5.22 ms, total: 129 ms
Wall time: 331 ms


<xarray.Dataset>
Dimensions:       (lat: 192, zlon: 1, nbnd: 2, lon: 288, lev: 70, ilev: 71, time: 2940)
Coordinates:
  * lat           (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * zlon          (zlon) float64 0.0
  * lon           (lon) float64 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lev           (lev) float64 5.96e-06 9.827e-06 1.62e-05 ... 976.3 992.6
  * ilev          (ilev) float64 4.5e-06 7.42e-06 1.223e-05 ... 985.1 1e+03
  * time          (time) object 2035-02-01 00:00:00 ... 2071-01-01 00:00:00
Dimensions without coordinates: nbnd
Data variables: (12/27)
    zlon_bnds     (zlon, nbnd) float64 dask.array<chunksize=(1, 2), meta=np.ndarray>
    gw            (lat) float64 dask.array<chunksize=(192,), meta=np.ndarray>
    hyam          (lev) float64 dask.array<chunksize=(70,), meta=np.ndarray>
    hybm          (lev) float64 dask.array<chunksize=(70,), meta=np.ndarray>
    P0            float64 ...
    hyai          (ilev) float64 dask.array<chunksize=(71,), meta=np.ndarray>
    ...            ...
    n2ovmr        (time) float64 dask.array<chunksize=(24,), meta=np.ndarray>
    f11vmr        (time) float64 dask.array<chunksize=(24,), meta=np.ndarray>
    f12vmr        (time) float64 dask.array<chunksize=(24,), meta=np.ndarray>
    sol_tsi       (time) float64 dask.array<chunksize=(24,), meta=np.ndarray>
    nsteph        (time) int32 dask.array<chunksize=(24,), meta=np.ndarray>
    T             (time, lev, lat, lon) float32 dask.array<chunksize=(24, 70, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.001
    logname:           geostrat
    host:              cheyenne5
    initial_file:      b.e21.BWSSP245cmip6.f09_g17.CMIP6-SSP2-4.5-WACCM.001.c...
    topography_file:   /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/fv_0.9x1...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  month_1

In [30]:
%%time
ds2.mean(dim='time').compute()

CPU times: user 3.86 s, sys: 219 ms, total: 4.08 s
Wall time: 11.6 s


<xarray.Dataset>
Dimensions:    (lat: 192, zlon: 1, nbnd: 2, lon: 288, lev: 70, ilev: 71)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * zlon       (zlon) float64 0.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lev        (lev) float64 5.96e-06 9.827e-06 1.62e-05 ... 957.5 976.3 992.6
  * ilev       (ilev) float64 4.5e-06 7.42e-06 1.223e-05 ... 967.5 985.1 1e+03
Dimensions without coordinates: nbnd
Data variables: (12/24)
    zlon_bnds  (zlon, nbnd) float64 0.0 358.8
    gw         (lat) float64 3.382e-05 0.0002705 ... 0.0002705 3.382e-05
    hyam       (lev) float64 5.96e-09 9.827e-09 1.62e-08 ... 0.001989 0.0
    hybm       (lev) float64 0.0 0.0 0.0 0.0 0.0 ... 0.9251 0.9512 0.9743 0.9926
    P0         float64 1e+05
    hyai       (ilev) float64 4.5e-09 7.42e-09 1.223e-08 ... 0.003979 0.0 0.0
    ...         ...
    n2ovmr     float64 3.573e-07
    f11vmr     float64 1.347e-10
    f12vmr     float64 3.588e-10
    sol_tsi    float64 -1.0
    nsteph     float64 2.843e+05
    T          (lev, lat, lon) float32 608.4 608.4 608.4 ... 260.1 260.1 260.1